#Setup Google Gemini - API

In [ ]:
!pip install -U google-generativeai

In [ ]:
import os

# Replace with your actual key
os.environ["GEMINI_API_KEY"] = "your_api"
from google import genai

# Initialize the client — it will automatically use GEMINI_API_KEY
client = genai.Client()

#Setup Open AI GPT - API


In [ ]:
!pip install openai

In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "your_api"
client = OpenAI()

# Testing the Models’ Comprehension of Greek Phrases Using OpenAI’s Two Embedding Models

## OpenAI Embeddings — Model: text-embedding-3-large

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import plotly.express as px


def embed_texts_openai(texts, model="text-embedding-3-large"):
    """
    Παίρνει λίστα από κείμενα και επιστρέφει embeddings (λίστα από λίστες floats)
    """
    response = client.embeddings.create(
        model=model,
        input=texts
    )
    return [d.embedding for d in response.data]



import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import plotly.express as px

def plot_words_2d_openai(words, query_word, model="text-embedding-3-large"):
    """
    Create a clean 2D PCA scatter plot of OpenAI embeddings.
    Highlights the query word distinctly.
    """
    # 1) Get embeddings
    all_texts = words + [query_word]
    embs = embed_texts_openai(all_texts, model=model)
    E = np.vstack(embs)

    # 2) PCA → 2D
    pca = PCA(n_components=2, random_state=42)
    X2 = pca.fit_transform(E)

    # 3) Build dataframe for plotting
    df_plot = pd.DataFrame({
        "x": X2[:, 0],
        "y": X2[:, 1],
        "Label": words + [query_word],
        "Type": ["Keyword"] * len(words) + ["Query"]
    })

    # 4) Create the figure
    fig = px.scatter(
        df_plot,
        x="x", y="y",
        text="Label",
        color="Type",
        color_discrete_map={"Keyword": "#1f77b4", "Query": "#d62728"},
        symbol="Type",
        size=[12 if t=="Keyword" else 28 for t in df_plot["Type"]],
        hover_name="Label",
        title=f"2D Embedding Map ({model})",
        template="plotly_white",
    )

    # 5) Style updates
    fig.update_traces(
        textposition="top center",
        marker=dict(opacity=0.85, line=dict(width=1, color="DarkSlateGrey")),
        textfont=dict(size=10)
    )

    fig.update_layout(
        title=dict(x=0.5, xanchor="center", font=dict(size=18, family="Arial")),
        legend=dict(
            title="Item Type",
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5,
            font=dict(size=12)
        ),
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        width=700,
        height=600
    )

    fig.show()
    return fig, df_plot

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA

words = ['Οικονομικό έγκλημα', 'Απάτη ΦΠΑ', 'Νίκος Κοκλώνης', 'Αχαρναϊκός', 'Ελληνικό FBI']
query = "ποδοσφαιρο"

_ = plot_words_2d_openai(words, query,model="text-embedding-3-large")

## OpenAI Embeddings — Model: text-embedding-ada-002

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA

words = ['Οικονομικό έγκλημα', 'Απάτη ΦΠΑ', 'Νίκος Κοκλώνης', 'Αχαρναϊκός', 'Ελληνικό FBI']
query = "ποδοσφαιρο"

_ = plot_words_2d_openai(words, query,model="text-embedding-ada-002")

## Gemini Embeddings — Model: gemini-embedding-001

In [ ]:
import numpy as np
import google.generativeai as genai

# Make sure you've already configured your key somewhere:
# genai.configure(api_key="YOUR_GEMINI_API_KEY")

def embed_texts_gemini(texts,
                       model="models/text-embedding-004",
                       task_type="RETRIEVAL_DOCUMENT",
                       normalize=True):
    """
    Compatibility wrapper for Gemini embeddings across SDK variants.
    Returns: list[list[float]]
    """
    embs = []

    # Try to prepare a client (some SDK variants expose top-level funcs, some via client)
    client = None
    try:
        client = genai.Client()
    except Exception:
        client = None

    for t in texts:
        resp = None

        # 1) Most common (top-level API)
        if hasattr(genai, "embed_content"):
            try:
                resp = genai.embed_content(model=model, content=t, task_type=task_type)
            except Exception:
                resp = None

        # 2) Newer client-level API
        if resp is None and client is not None and hasattr(client, "embed_content"):
            try:
                resp = client.embed_content(model=model, content=t, task_type=task_type)
            except Exception:
                resp = None

        # If still none, give a meaningful error
        if resp is None:
            raise AttributeError(
                "Gemini embed API not found on this SDK. "
                "Try updating: `pip install -U google-generativeai` "
                "and ensure `genai.configure(api_key=...)` ran."
            )

        # ---- Extract the vector robustly across response shapes ----
        if isinstance(resp, dict) and "embedding" in resp:
            vec = resp["embedding"]
        elif hasattr(resp, "embedding"):
            vec = resp.embedding
        elif hasattr(resp, "embeddings"):
            # sometimes comes as a list of embeddings
            v = resp.embeddings[0]
            vec = v.values if hasattr(v, "values") else v
        else:
            raise ValueError(f"Unexpected embedding response shape: {type(resp)} / keys: {getattr(resp, 'keys', lambda:[])()}")

        vec = np.array(vec, dtype="float32")
        if normalize:
            norm = np.linalg.norm(vec) + 1e-12
            vec = vec / norm
        embs.append(vec.tolist())

    return embs


In [ ]:
def plot_words_2d_generic(words, query_word, embed_func, title_suffix=""):
    """
    embed_func: callable(texts:list[str]) -> list[list[float]]
    """
    # 1) embeddings
    all_texts = words + [query_word]
    embs = embed_func(all_texts)
    E = np.vstack(embs)

    # 2) PCA → 2D
    pca = PCA(n_components=2, random_state=42)
    X2 = pca.fit_transform(E)

    # 3) build plot df
    df_plot = pd.DataFrame({
        "x": X2[:, 0],
        "y": X2[:, 1],
        "Label": words + [query_word],
        "Type": ["Keyword"] * len(words) + ["Query"]
    })

    # 4) plot (clean aesthetics; query stands out)
    fig = px.scatter(
        df_plot,
        x="x", y="y",
        text="Label",
        color="Type",
        color_discrete_map={"Keyword": "#1f77b4", "Query": "#d62728"},
        symbol="Type",
        size=[12 if t=="Keyword" else 28 for t in df_plot["Type"]],
        hover_name="Label",
        title=f"2D Embedding Map {title_suffix}".strip(),
        template="plotly_white",
    )
    fig.update_traces(
        textposition="top center",
        marker=dict(opacity=0.9, line=dict(width=1, color="DarkSlateGrey")),
        textfont=dict(size=10)
    )
    fig.update_layout(
        title=dict(x=0.5, xanchor="center", font=dict(size=18, family="Arial")),
        legend=dict(
            title="Item Type",
            orientation="h",
            yanchor="bottom", y=-0.2,
            xanchor="center", x=0.5,
            font=dict(size=12)
        ),
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        width=740, height=620
    )
    fig.show()
    return fig, df_plot


def plot_words_2d_gemini(words, query_word,
                         model="text-embedding-004",
                         task_type="SEMANTIC_SIMILARITY"):
    return plot_words_2d_generic(
        words, query_word,
        embed_func=lambda texts: embed_texts_gemini(texts, model=model, task_type=task_type),
        title_suffix=f"(Gemini • {model.split('/')[-1]})"
    )

In [ ]:
words = ['Οικονομικό έγκλημα', 'Απάτη ΦΠΑ', 'Νίκος Κοκλώνης', 'Αχαρναϊκός', 'Ελληνικό FBI']
query = "ποδοσφαιρο"


plot_words_2d_gemini(words, query, model="gemini-embedding-001")


(Figure({
     'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>Typ' ... 'r>Label=%{text}<extra></extra>'),
               'hovertext': array(['Οικονομικό έγκλημα', 'Απάτη ΦΠΑ', 'Νίκος Κοκλώνης', 'Αχαρναϊκός',
                                   'Ελληνικό FBI'], dtype=object),
               'legendgroup': 'Keyword',
               'marker': {'color': '#1f77b4',
                          'line': {'color': 'DarkSlateGrey', 'width': 1},
                          'opacity': 0.9,
                          'size': array([12, 12, 12, 12, 12]),
                          'sizemode': 'area',
                          'sizeref': 0.07,
                          'symbol': 'circle'},
               'mode': 'markers+text',
               'name': 'Keyword',
               'orientation': 'v',
               'showlegend': True,
               'text': array(['Οικονομικό έγκλημα', 'Απάτη ΦΠΑ', 'Νίκος Κοκλώνης', 'Αχαρναϊκός',
                              'Ελληνικό FBI'], dtype=object),
            

## Gemini Embeddings — Model: text-embedding-004

In [ ]:
plot_words_2d_gemini(words, query, model="text-embedding-004")

(Figure({
     'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>Typ' ... 'r>Label=%{text}<extra></extra>'),
               'hovertext': array(['Οικονομικό έγκλημα', 'Απάτη ΦΠΑ', 'Νίκος Κοκλώνης', 'Αχαρναϊκός',
                                   'Ελληνικό FBI'], dtype=object),
               'legendgroup': 'Keyword',
               'marker': {'color': '#1f77b4',
                          'line': {'color': 'DarkSlateGrey', 'width': 1},
                          'opacity': 0.9,
                          'size': array([12, 12, 12, 12, 12]),
                          'sizemode': 'area',
                          'sizeref': 0.07,
                          'symbol': 'circle'},
               'mode': 'markers+text',
               'name': 'Keyword',
               'orientation': 'v',
               'showlegend': True,
               'text': array(['Οικονομικό έγκλημα', 'Απάτη ΦΠΑ', 'Νίκος Κοκλώνης', 'Αχαρναϊκός',
                              'Ελληνικό FBI'], dtype=object),
            

# Summarizing Results

It is clear that the most comprehensive and high-quality embedding representations come from text-embedding-3-large and gemini-embedding-001. We will continue with the former, as it also appears to perform better for clustering similar phrases.

#AI-Powered Semantic Article Search Using FAISS with HNSW Indexing

In [ ]:
!pip install faiss-cpu numpy pandas tiktoken rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 80.1 MB/s eta 0:00:00


## Loading Dataset - Here’s an Example for my Own Dataset

In [ ]:
import pandas as pd
excel_path = "/content/Content Clusters.xlsx"

In [ ]:
import pandas as pd

C_CLUSTER = "Cluster"
C_TOPICS  = "Κεντρικά Topics"

xls = pd.ExcelFile(excel_path)
frames = []

for sheet in xls.sheet_names:
    # διαβάζουμε το tab (header=1 όπως είχες), αφήνουμε να φορτώσει όλο
    df = pd.read_excel(excel_path, sheet_name=sheet, header=1)
    if C_CLUSTER not in df.columns or C_TOPICS not in df.columns:
        continue

    # forward-fill για merged cells
    df["cluster"] = df[C_CLUSTER].ffill()

    # καθάρισμα topics: strip, drop κενά/“nan”
    s = df[C_TOPICS].astype(str).str.strip()
    s = s[(s.str.len() > 0) & (s.str.lower() != "nan")]
    df = pd.DataFrame({"cluster": df["cluster"], C_TOPICS: s}).dropna(subset=[C_TOPICS])

    # groupby χωρίς sort, unique με διατήρηση σειράς (pd.unique)
    grp = (
        df.groupby("cluster", sort=False)[C_TOPICS]
          .apply(lambda s: list(pd.unique(pd.Series([t for t in s if t.strip()]))))
          .reset_index(name="keywords")
    )

    grp.insert(0, "category", sheet)  # βάζουμε category = όνομα sheet
    frames.append(grp)

df_all1 = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["category","cluster","keywords"])
print(df_all1.head(20))


          category                                          cluster  \
0       ΕΚΠΑΙΔΕΥΣΗ                                      Πανελλήνιες   
1       ΕΚΠΑΙΔΕΥΣΗ                         Προετοιμασία Πανελληνίων   
2       ΕΚΠΑΙΔΕΥΣΗ  Επαγγελματικός Προσανατολισμός & Επιλογή Σχολών   
3       ΕΚΠΑΙΔΕΥΣΗ                  Εκπαιδευτικά Νέα & Ανακοινώσεις   
4       ΕΚΠΑΙΔΕΥΣΗ           Μεταπτυχιακά και Σπουδές στο Εξωτερικό   
5       ΕΚΠΑΙΔΕΥΣΗ                          Σεμινάρια & Soft Skills   
6       ΕΚΠΑΙΔΕΥΣΗ        Διαχείριση Τάξης και Παιδαγωγικές Μέθοδοι   
7       ΕΚΠΑΙΔΕΥΣΗ              Ψηφιακά Εργαλεία για Εκπαιδευτικούς   
8        ΟΙΚΟΝΟΜΙΑ                               Ελληνική Οικονομία   
9        ΟΙΚΟΝΟΜΙΑ                          Συντάξεις & Ασφαλιστικό   
10       ΟΙΚΟΝΟΜΙΑ                                Διεθνής Οικονομία   
11       ΟΙΚΟΝΟΜΙΑ                Επιχειρήσεις & Επιχειρηματικότητα   
12       ΟΙΚΟΝΟΜΙΑ                              Αγορές & Επενδύσεις   
13    

NameError: name 'all_keywords' is not defined

##Data Management

In [ ]:
import pandas as pd

SPECIAL_TABS = [
    "ΚΑΙΡΟΣ", "ΕΥΡΩΕΚΛΟΓΕΣ", "ΑΡΓΙΕΣ", "ΑΠΕΡΓΙΕΣ",
    "ΠΑΝΕΛΛΗΝΙΕΣ", "ΣΥΝΤΑΞΕΙΣ", "ΚΟΙΝΩΝΙΚΟΣ ΤΟΥΡΙΣΜΟΣ",
]

def unique_preserve_order(seq):
    seen, out = set(), []
    for x in seq:
        x = "" if pd.isna(x) else str(x).strip()
        if not x or x.lower() == "nan":
            continue
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

def parse_wide_sheet(excel_path: str, sheet: str) -> pd.DataFrame:
    """2η γραμμή = τίτλοι clusters ανά στήλη, από 3η = keywords κάτω από κάθε cluster."""
    usecols = "A:C" if sheet.upper().strip() == "ΕΥΡΩΕΚΛΟΓΕΣ" else None
    df_raw = pd.read_excel(excel_path, sheet_name=sheet, header=None, usecols=usecols)

    # πρέπει να υπάρχουν τουλάχιστον 2 σειρές (τίτλοι clusters στη 2η)
    if df_raw.shape[0] < 2 or df_raw.shape[1] == 0:
        return pd.DataFrame(columns=["category", "cluster", "keywords"])

    cluster_row = df_raw.iloc[1].tolist()  # 2η γραμμή = ονόματα cluster ανά στήλη
    rows = []

    # διατρέχουμε κάθε στήλη 1-1 (ασφαλές σε NaN/κενά/διπλότυπα headers)
    for col_idx, cluster_name in enumerate(cluster_row):
        cluster = "" if pd.isna(cluster_name) else str(cluster_name).strip()
        if not cluster or cluster.lower() == "nan":
            continue  # αγνοούμε στήλες χωρίς έγκυρο τίτλο

        # keywords = όλες οι τιμές από 3η γραμμή και κάτω στην ίδια στήλη
        col_vals = df_raw.iloc[2:, col_idx].tolist() if col_idx < df_raw.shape[1] else []
        keywords = unique_preserve_order(col_vals)
        if not keywords:
            continue

        rows.append({"category": sheet, "cluster": cluster, "keywords": keywords})

    # σταθερό schema σε κάθε περίπτωση
    return pd.DataFrame(rows, columns=["category", "cluster", "keywords"])

# === Τρέχουμε για όλα τα ειδικά tabs και τα ενώνουμε ===
frames = []
for tab in SPECIAL_TABS:
    try:
        out = parse_wide_sheet(excel_path, tab)
        if not out.empty:
            frames.append(out)
    except Exception as e:
        print(f"[WARN] Πρόβλημα στο sheet '{tab}': {e}")

df_all2 = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["category","cluster","keywords"])
print("Rows parsed:", len(df_all2))
print(df_all2.head(10))


Rows parsed: 27
      category                     cluster  \
0       ΚΑΙΡΟΣ            General Keywords   
1       ΚΑΙΡΟΣ          Καιρός Ανά Περιοχή   
2       ΚΑΙΡΟΣ             Πρόγνωση Καιρού   
3  ΕΥΡΩΕΚΛΟΓΕΣ            Ευρωεκλογές 2024   
4  ΕΥΡΩΕΚΛΟΓΕΣ  Ευρωεκλογές 2024 Υποψήφιοι   
5  ΕΥΡΩΕΚΛΟΓΕΣ               Ευρωβουλευτές   
6       ΑΡΓΙΕΣ                 Πρωτοχρονιά   
7       ΑΡΓΙΕΣ                   Θεοφάνεια   
8       ΑΡΓΙΕΣ              Καθαρά Δευτέρα   
9       ΑΡΓΙΕΣ                 25η Μαρτίου   

                                            keywords  
0  [καιρόσ, ο καιρόσ αυριο, καιρόσ σήμερα, εμυ κα...  
1  [καιροσ αθηνα, καιροσ θεσσαλονικη, καιροσ πατρ...  
2  [προγνωση καιρου, μακροπροθεσμη προγνωση καιρο...  
3  [ευρωεκλογες 2024 αποτελεσματα, πότε είναι οι ...  
4  [Ευρωεκλογές 2024 υποψήφιοι νδ, Ευρωεκλογές 20...  
5  [ευρωβουλευτεσ ελλαδα, ευρωβουλευτεσ κυπρου, ε...  
6  [ευχες για πρωτοχρονια, κινεζικη πρωτοχρονια π...  
7  [τι συμβολιζουν τα θεοφανεια, τι γ

In [ ]:
import pandas as pd

def unique_preserve_order(seq):
    seen, out = set(), []
    for x in seq:
        x = "" if pd.isna(x) else str(x).strip()
        if not x or x.lower() == "nan":
            continue
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

def read_range(excel_path: str, sheet: str, usecols: str, start_row: int, end_row: int):
    """Διαβάζει range (χωρίς headers)."""
    nrows = max(0, end_row - start_row + 1)
    return (
        pd.read_excel(excel_path, sheet_name=sheet, header=None,
                      usecols=usecols, skiprows=start_row - 1, nrows=nrows)
        if nrows > 0 else pd.DataFrame()
    )

def parse_block_to_long(df_block, category_name):
    """1η γραμμή = clusters, από 2η = keywords."""
    if df_block.empty:
        return pd.DataFrame(columns=["category", "cluster", "keywords"])
    rows = []
    for i, c in enumerate(df_block.iloc[0]):
        cluster = "" if pd.isna(c) else str(c).strip()
        if not cluster or cluster.lower() == "nan":
            continue
        kws = unique_preserve_order(df_block.iloc[1:, i].tolist())
        if kws:
            rows.append({"category": category_name, "cluster": cluster, "keywords": kws})
    return pd.DataFrame(rows, columns=["category", "cluster", "keywords"])

# --------------------------
# ΦΑΓΗΤΟ: δύο blocks
# --------------------------
def parse_food_sheet(excel_path):
    sheet = "ΦΑΓΗΤΟ"
    b1 = parse_block_to_long(read_range(excel_path, sheet, "A:G", 2, 22), sheet)
    b2 = parse_block_to_long(read_range(excel_path, sheet, "A:F", 26, 43), sheet)
    frames = [x for x in [b1, b2] if not x.empty]
    if not frames:
        return pd.DataFrame(columns=["category", "cluster", "keywords"])
    df = pd.concat(frames, ignore_index=True)
    df["keywords"] = df["keywords"].apply(unique_preserve_order)
    return df

# --------------------------
# ΜΟΔΑ: ένας block
# --------------------------
def parse_fashion_sheet(excel_path):
    sheet = "ΜΟΔΑ"
    df = parse_block_to_long(read_range(excel_path, sheet, "A:C", 2, 25), sheet)
    if df.empty:
        return pd.DataFrame(columns=["category", "cluster", "keywords"])
    df["keywords"] = df["keywords"].apply(unique_preserve_order)
    return df

# === Τρέξιμο ===
df_all3 = parse_food_sheet(excel_path)
df_fashion = parse_fashion_sheet(excel_path)
df_all3_fashion = pd.concat([df_all3, df_fashion], ignore_index=True)

print("ΦΑΓΗΤΟ rows:", len(df_all3))
print(df_all3.head(10))
print("\nΜΟΔΑ rows:", len(df_fashion))
print(df_fashion.head(10))
print("\nΣΥΝΟΛΟ rows:", len(df_all3_fashion))


ΦΑΓΗΤΟ rows: 13
  category                cluster  \
0   ΦΑΓΗΤΟ             Πρωτοχονιά   
1   ΦΑΓΗΤΟ           Τσικνοπέμπτη   
2   ΦΑΓΗΤΟ         Καθαρά Δευτέρα   
3   ΦΑΓΗΤΟ  Σαρακοστή - Νηστίσιμα   
4   ΦΑΓΗΤΟ                  Πάσχα   
5   ΦΑΓΗΤΟ              Καλοκαίρι   
6   ΦΑΓΗΤΟ           Χριστούγεννα   
7   ΦΑΓΗΤΟ                  Κρέας   
8   ΦΑΓΗΤΟ       Ψάρι - Θαλασσινά   
9   ΦΑΓΗΤΟ               Ζυμαρικά   

                                            keywords  
0  [ιδεες για πρωτοχρονιατικο τραπεζι, πρωτοχρονι...  
1  [μενου τσικνοπεμπτης, φαγητα για τσικνοπεμπτη,...  
2  [καθαρα δευτερα μενου, καθαρα δευτερα φαγητο, ...  
3  [σαρακοστιανα φαγητα, νηστισιμα γλυκα, χταποδι...  
4  [πασχαλινο τραπεζι, σαλατες για πασχαλινο τραπ...  
5  [καλοκαιρινα φαγητα, γεμιστα, λαδερά, γλυκό ψυ...  
6  [χριστουγεννιατικο τραπεζι, χριστουγεννιατικη ...  
7  [τηγανια χοιρινο, χοιρινο στο φουρνο, χοιρινο ...  
8  [γαριδεσ σαγανακι, καλαμαρακια τηγανητα, κριθα...  
9  [ριζοτο με λαχανικα, κα

In [ ]:
import pandas as pd

# Υποθέτουμε ότι τα έχεις ήδη έτοιμα: df_all1, df_all2, df_all3

df_final = pd.concat([df_all1, df_all2, df_all3], ignore_index=True)

print(df_final.shape)
print(df_final.head())


(100, 3)
     category                                          cluster  \
0  ΕΚΠΑΙΔΕΥΣΗ                                      Πανελλήνιες   
1  ΕΚΠΑΙΔΕΥΣΗ                         Προετοιμασία Πανελληνίων   
2  ΕΚΠΑΙΔΕΥΣΗ  Επαγγελματικός Προσανατολισμός & Επιλογή Σχολών   
3  ΕΚΠΑΙΔΕΥΣΗ                  Εκπαιδευτικά Νέα & Ανακοινώσεις   
4  ΕΚΠΑΙΔΕΥΣΗ           Μεταπτυχιακά και Σπουδές στο Εξωτερικό   

                                            keywords  
0  [Πρόγραμμα Πανελληνίων 2025, Μηχανογραφικό, Βά...  
1  [Οργάνωση διαβάσματος, Συμβουλές για μείωση άγ...  
2  [Πώς να επιλέξεις τη σωστή σχολή, Αναλυτική πα...  
3  [Αλλαγές εξεταστικού συστήματος, Εκπαιδευτική ...  
4  [Σπουδές στην Ευρώπη: κόστος, προϋποθέσεις και...  


In [ ]:
df_final["cluster"]
all_keywords = df_final["cluster"].tolist()
import ast

## Graph's Parameters

| **Parameter**                   | **What It Does**                                                                | **What It Affects**                                                                                                            |
| ------------------------------- | ------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------ |
| **`M`**                         | Maximum number of **neighbors** each node keeps in the graph (typically 16–64). | 🔹 Higher `M` → more accurate results, slower index build, higher memory usage.<br>🔹 Lower `M` → faster build, less accuracy. |
| **`efConstruction`**            | Number of candidate neighbors considered during graph construction.             | 🔹 Higher `efConstruction` → better connection quality (more precise graph).<br>🔹 But increases build time.                   |
| **`efSearch`**                  | Number of nodes explored during a search query.                                 | 🔹 Higher `efSearch` → higher accuracy, slower search.<br>🔹 Lower `efSearch` → faster but coarser search results.             |
| **`metric`**                    | Distance metric: `faiss.METRIC_INNER_PRODUCT` or `faiss.METRIC_L2`.             | 🔹 If you normalize embeddings, **INNER_PRODUCT ≈ cosine similarity**.                                                         |
| **`normalize`**                 | Whether to normalize vectors to unit length.                                    | 🔹 Makes semantic search results more stable and consistent.                                                                   |
| **`BATCH`** (in embedding loop) | Batch size used for API calls during embedding generation.                      | 🔹 Pure runtime/memory trade-off — larger batches are faster but use more memory.                                              |


| **Use Case**                               | **Recommended Parameters**                               |
| ------------------------------------------ | -------------------------------------------------------- |
| 🔹 **Small corpus (up to ~100K keywords)** | `HNSWFlat`, `M=32`, `efConstruction=100`, `efSearch=32`  |
| 🔹 **Medium corpus (500K–1M)**             | `HNSWFlat`, `M=64`, `efConstruction=200`, `efSearch=64`  |
| 🔹 **Very large corpus (10M+)**            | `IVFPQ` or `IVFSQ`, `nlist=4096+`, `nprobe=64`           |
| 🔹 **Memory constraints**                  | Use PQ compression (e.g., 8 bytes per vector)            |
| 🔹 **Maximum accuracy, small corpus**      | `FlatL2` (brute-force search) — no ANN, best for testing |


In [ ]:
# semantic_search_from_article.py
# Build a FAISS HNSW index from all_keywords and retrieve nearest neighbors
# given a full article text (semantic search).

import os, uuid, re
from pathlib import Path
import numpy as np
import pandas as pd
from openai import OpenAI
import faiss

# ============ CONFIG ============
EMBED_MODEL = "text-embedding-3-large"   # 3072-dim
DIM = 3072
BATCH = 128
NORMALIZE = True
FORCE_REBUILD = False

INDEX_DIR = Path("vector_index_keywords_only")
INDEX_DIR.mkdir(exist_ok=True)
INDEX_PATH = INDEX_DIR / "hnsw.index"
META_PATH  = INDEX_DIR / "meta.parquet"


# ============ HELPERS ============
def normalize_rows(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    return x / n

def embed_texts(texts):
    """Batch embeddings for a list of short texts (keywords)."""
    embs = []
    for i in range(0, len(texts), BATCH):
        chunk = texts[i:i+BATCH]
        resp = client.embeddings.create(model=EMBED_MODEL, input=chunk)
        embs.extend([e.embedding for e in resp.data])
    X = np.array(embs, dtype="float32")
    return normalize_rows(X) if NORMALIZE else X

def _flatten_keywords(kws):
    """Flatten + dedupe (preserve order) + strip."""
    flat = []
    for item in kws:
        if isinstance(item, (list, tuple, set)):
            flat.extend(list(item))
        else:
            flat.append(item)
    flat = [str(x).strip() for x in flat if str(x).strip() != ""]
    seen, out = set(), []
    for x in flat:
        if x not in seen:
            out.append(x); seen.add(x)
    return out

def _clean_text(t: str) -> str:
    """Very light cleanup for article text."""
    t = re.sub(r"\s+", " ", t).strip()
    return t

# ============ CORPUS ============
def load_corpus_from_keywords(all_keywords):
    kws = _flatten_keywords(all_keywords)
    if not kws:
        raise ValueError("Δώσε μη κενή λίστα all_keywords.")
    df = pd.DataFrame({"campaign_title": kws})
    df["doc"] = df["campaign_title"]
    df["doc_id"] = [str(uuid.uuid4()) for _ in range(len(df))]
    df["url"] = None
    df["category"] = None
    return df[["doc_id","campaign_title","url","category","doc"]]

# ============ INDEX BUILD / LOAD ============
def _delete_old_index_files():
    try: INDEX_PATH.unlink(missing_ok=True)
    except Exception: pass
    try: META_PATH.unlink(missing_ok=True)
    except Exception: pass

def build_or_load_index(all_keywords):
    """Builds index from keywords if needed; otherwise loads from disk."""
    if FORCE_REBUILD and (INDEX_PATH.exists() or META_PATH.exists()):
        print(">> FORCE_REBUILD=True → deleting old index files…")
        _delete_old_index_files()

    if INDEX_PATH.exists() and META_PATH.exists():
        print(">> Loading existing index & metadata…")
        idx = faiss.read_index(str(INDEX_PATH))
        meta = pd.read_parquet(META_PATH)
        if not {"campaign_title","doc"}.issubset(meta.columns):
            print("!! Old/invalid meta schema. Rebuilding…")
            _delete_old_index_files()
        else:
            return idx, meta

    print(">> Building index from scratch (keywords only)…")
    df = load_corpus_from_keywords(all_keywords)
    texts = df["doc"].tolist()

    print(f">> Embedding {len(texts)} keywords…")
    X = embed_texts(texts)

    # HNSW index (ANN)
    m = 64
    idx = faiss.IndexHNSWFlat(DIM, m, faiss.METRIC_INNER_PRODUCT)
    idx.hnsw.efConstruction = 200
    idx.hnsw.efSearch = 64

    print(">> Adding vectors to HNSW index…")
    idx.add(X)

    faiss.write_index(idx, str(INDEX_PATH))
    df.to_parquet(META_PATH, index=False)
    print(">> Done. Saved index & metadata.")
    return idx, df

# ============ EMBEDDING (QUERY / DOCUMENT) ============
def embed_query(q: str, translate_to_en: bool = False) -> np.ndarray:
    text = _clean_text(q)
    if translate_to_en:
        t = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":f"Translate to concise English suitable for semantic search:\n\n{text}"}],
            temperature=0.0
        )
        text = t.choices[0].message.content.strip() or text
    e = client.embeddings.create(model=EMBED_MODEL, input=[text]).data[0].embedding
    v = np.array([e], dtype="float32")
    return normalize_rows(v) if NORMALIZE else v

def embed_document(article_text: str, summarize: bool = True) -> np.ndarray:
    """
    Turn a full article into one embedding vector.
    summarize=True: brief summary first to denoise long texts.
    """
    text = _clean_text(article_text)
    if summarize:
        summ = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{
                "role":"user",
                "content":(
                    "Δώσε πολύ σύντομη περίληψη (<=80 λέξεις) "
                    "με τις βασικές έννοιες για semantic search:\n\n" + text
                )
            }],
            temperature=0.0
        ).choices[0].message.content.strip()
        base = summ if summ else text
    else:
        base = text

    e = client.embeddings.create(model=EMBED_MODEL, input=[base]).data[0].embedding
    v = np.array([e], dtype="float32")
    return normalize_rows(v) if NORMALIZE else v

# ============ SEARCH ============
def search(q: str, k: int = 6, translate_to_en: bool = False):
    """
    Classic semantic search over the keywords using a short query string.
    """
    global index, meta
    try:
        index, meta
    except NameError:
        raise RuntimeError("Κάλεσε πρώτα build_or_load_index(all_keywords).")

    qv = embed_query(q, translate_to_en=translate_to_en)
    topn = min(max(k*5, k), len(meta))
    D, I = index.search(qv, topn)

    cand = meta.iloc[I[0]].copy()
    cand["score_vec"] = D[0]

    # lightweight keyword boost (optional)
    from rapidfuzz import fuzz
    cand["keyword_boost"] = cand["campaign_title"].fillna("").apply(
        lambda t: fuzz.token_set_ratio(q, str(t))/100.0
    )
    cand["final_score"] = 0.9*cand["score_vec"] + 0.1*cand["keyword_boost"]
    return cand.sort_values("final_score", ascending=False).head(k)[
        ["doc_id","campaign_title","url","category","final_score"]
    ]

def search_from_article(article_text: str, k: int = 10, summarize: bool = True):
    """
    Give a FULL ARTICLE; returns the k nearest keywords from the graph.
    """
    global index, meta
    try:
        index, meta
    except NameError:
        raise RuntimeError("Κάλεσε πρώτα build_or_load_index(all_keywords).")

    qv = embed_document(article_text, summarize=summarize)
    topn = min(max(k*5, k), len(meta))
    D, I = index.search(qv, topn)

    cand = meta.iloc[I[0]].copy()
    cand["score_vec"] = D[0]

    # optional small boost vs title text
    from rapidfuzz import fuzz
    cand["keyword_boost"] = cand["campaign_title"].fillna("").apply(
        lambda t: fuzz.token_set_ratio(article_text, str(t))/100.0
    )
    cand["final_score"] = 0.9*cand["score_vec"] + 0.1*cand["keyword_boost"]
    return cand.sort_values("final_score", ascending=False).head(k)[
        ["doc_id","campaign_title","url","category","final_score"]
    ]


# --- στο script σου (δίπλα στα search / embed_document) ---

def search_from_article(article_text: str, k: int = 10, summarize: bool = True):
    """
    Δίνεις ΠΛΗΡΕΣ άρθρο και επιστρέφει τα k κοντινότερα keywords από τον γράφο.
    summarize=True: κάνει μικρή περίληψη για πιο “καθαρό” embedding.
    """
    global index, meta
    try:
        index, meta  # πρέπει να έχεις ήδη καλέσει build_or_load_index(ALL_KEYWORDS)
    except NameError:
        raise RuntimeError("Κάλεσε πρώτα build_or_load_index(all_keywords).")

    # ✳️ embed του άρθρου (με προαιρετική περίληψη)
    text = re.sub(r"\s+", " ", article_text).strip()
    if summarize:
        summ = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{
                "role": "user",
                "content": "Δώσε πολύ σύντομη περίληψη (<=80 λέξεις) με τις βασικές έννοιες για semantic search:\n\n" + text
            }],
            temperature=0.0
        ).choices[0].message.content.strip()
        base = summ or text
    else:
        base = text

    e = client.embeddings.create(model=EMBED_MODEL, input=[base]).data[0].embedding
    qv = np.array([e], dtype="float32")
    if NORMALIZE:
        qv = qv / (np.linalg.norm(qv, axis=1, keepdims=True) + 1e-12)

    # ✳️ ANN search στον HNSW index
    topn = min(max(k*5, k), len(meta))   # πάρ’ το λίγο πιο πλατύ για μικρό rerank
    D, I = index.search(qv, topn)

    cand = meta.iloc[I[0]].copy()
    cand["score_vec"] = D[0]

    # μικρό keyword-boost για σταθερότητα
    from rapidfuzz import fuzz
    cand["keyword_boost"] = cand["campaign_title"].fillna("").apply(
        lambda t: fuzz.token_set_ratio(text, str(t))/100.0
    )
    cand["final_score"] = 0.9*cand["score_vec"] + 0.1*cand["keyword_boost"]

    out = cand.sort_values("final_score", ascending=False).head(k)
    return out[["campaign_title", "final_score"]]


In [ ]:
# 1) Χτίζεις/φορτώνεις index ΜΙΑ φορά
ALL_KEYWORDS = all_keywords
index, meta = build_or_load_index(ALL_KEYWORDS)

# 2) Δίνεις άρθρο & παίρνεις προτάσεις
article = """Οι ποινές για λογοκλοπή και χρήση βίας στα ΑΕΙ
Τι ορίζει η εγκύκλιος για τα παραπτώματα των φοιτητών - Το υπ. Παιδείας έχει δώσει στα ΑΕΙ προθεσμία έως τα τέλη Δεκεμβρίου για να επικαιροποιήσουν τα σχέδια για την ασφάλειά τους"""
suggested = search_from_article(article, k=10, summarize=False)
print(suggested.to_string(index=False))


>> Building index from scratch (keywords only)…
>> Embedding 97 keywords…
>> Adding vectors to HNSW index…
>> Done. Saved index & metadata.
                                      campaign_title  final_score
                     Εκπαιδευτικά Νέα & Ανακοινώσεις     0.359192
                            Προετοιμασία Πανελληνίων     0.298758
                 Ψηφιακά Εργαλεία για Εκπαιδευτικούς     0.295553
                                    Πανελλήνιες 2024     0.292335
                                         Πανελλήνιες     0.290243
     Επαγγελματικός Προσανατολισμός & Επιλογή Σχολών     0.283188
                  Internet, Δίκτυα & Κυβερνοασφάλεια     0.276783
           Διαχείριση Τάξης και Παιδαγωγικές Μέθοδοι     0.276720
Ελληνοτουρκικές Στρατηγικές και Στρατιωτική Ασφάλεια     0.268426
                                  Για τα επαγγέλματα     0.256646


## 3d Interactive Representation of Graph

In [ ]:
# hnsw_interactive_3d_keywords.py
# Interactive 3D visualization of your HNSW / k-NN graph over the keywords corpus.

import os
from pathlib import Path
import numpy as np
import pandas as pd

# Try FAISS; fallback if not present
HAS_FAISS = True
try:
    import faiss
except Exception:
    HAS_FAISS = False

from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import plotly.graph_objects as go

# ========= CONFIG =========
INDEX_DIR  = Path("vector_index_keywords_only")     # <-- ίδιο με το script σου
INDEX_PATH = INDEX_DIR / "hnsw.index"
META_PATH  = INDEX_DIR / "meta.parquet"

N_SAMPLES   = 500     # πλήθος κόμβων για το plot (200–600 για άνεση)
K_NEIGHBORS = 10      # ακμές ανά κόμβο
RANDOM_SEED = 42
TITLE_COL   = "campaign_title"  # hover label

# Fallback embeddings (αν δεν υπάρχει FAISS)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY")
EMBED_MODEL    = "text-embedding-3-large"

# ========= LOAD META =========
if not META_PATH.exists():
    raise FileNotFoundError(f"meta.parquet not found at {META_PATH}")
meta = pd.read_parquet(META_PATH)

if TITLE_COL not in meta.columns:
    # fallback hover label
    TITLE_COL = "doc" if "doc" in meta.columns else "doc_id"

# Sample subset για visualization
rng = np.random.default_rng(RANDOM_SEED)
N_total = len(meta)
N = min(N_total, N_SAMPLES)
sample_idx = np.array(sorted(rng.choice(N_total, size=N, replace=False))) if N_total > N else np.arange(N_total)

labels  = meta.iloc[sample_idx][TITLE_COL].astype(str).fillna("")
doc_ids = meta.iloc[sample_idx]["doc_id"].astype(str).fillna("")

# ========= VECTORS =========
if HAS_FAISS and INDEX_PATH.exists():
    # ----- Option A: πάρε τα vectors από τον FAISS index -----
    index = faiss.read_index(str(INDEX_PATH))
    ntotal = min(index.ntotal, len(meta))
    sample_idx = sample_idx[sample_idx < ntotal]

    D = index.d
    vecs = np.zeros((len(sample_idx), D), dtype="float32")
    tmp  = np.zeros((D,), dtype="float32")
    for i, fid in enumerate(sample_idx):
        index.reconstruct(int(fid), tmp)  # IndexHNSWFlat υποστηρίζει reconstruct
        vecs[i] = tmp

    # k-NN από το ίδιο το HNSW (για edges)
    D_knn, I_knn = index.search(vecs, K_NEIGHBORS)

else:
    # ----- Option B: δεν υπάρχει FAISS -> embeddings για το δείγμα -----
    print("FAISS not available or index file missing → fallback: fresh embeddings for the sample.")
    from openai import OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)

    texts = labels.tolist()
    BATCH = 128
    embs = []
    for i in range(0, len(texts), BATCH):
        chunk = texts[i:i+BATCH]
        resp = client.embeddings.create(model=EMBED_MODEL, input=chunk)
        embs.extend([e.embedding for e in resp.data])
    vecs = np.array(embs, dtype="float32")

    # cosine kNN πάνω στο δείγμα (για edges)
    knn = NearestNeighbors(n_neighbors=K_NEIGHBORS, metric="cosine")
    knn.fit(vecs)
    D_knn, I_knn = knn.kneighbors(vecs, n_neighbors=K_NEIGHBORS)

# Normalize για σταθερότητα στην προβολή
vecs = vecs / (np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12)

# ========= 3D PROJECTION (t-SNE) =========
tsne = TSNE(
    n_components=3,
    random_state=RANDOM_SEED,
    perplexity=min(30, max(5, len(vecs)//10)),
    learning_rate="auto",
    init="pca"
)
xyz = tsne.fit_transform(vecs)  # shape: (N, 3)

# ========= EDGES =========
if HAS_FAISS and INDEX_PATH.exists():
    # I_knn επιστρέφει global ids -> map σε local indices
    global_to_local = {int(gid): i for i, gid in enumerate(sample_idx)}
    edge_x, edge_y, edge_z = [], [], []
    for i_local, nbrs in enumerate(I_knn):
        src_global = int(sample_idx[i_local])
        for nbr_global in nbrs:
            nbr_global = int(nbr_global)
            if nbr_global == src_global:
                continue
            j_local = global_to_local.get(nbr_global, None)
            if j_local is None:
                continue
            x0, y0, z0 = xyz[i_local]
            x1, y1, z1 = xyz[j_local]
            edge_x += [x0, x1, None]
            edge_y += [y0, y1, None]
            edge_z += [z0, z1, None]
else:
    # local indices ήδη
    edge_x, edge_y, edge_z = [], [], []
    for i_local, nbrs in enumerate(I_knn):
        for j_local in nbrs:
            if j_local == i_local:
                continue
            x0, y0, z0 = xyz[i_local]
            x1, y1, z1 = xyz[int(j_local)]
            edge_x += [x0, x1, None]
            edge_y += [y0, y1, None]
            edge_z += [z0, z1, None]

# ========= PLOTLY 3D =========
hover_text = (labels + "<br><b>doc_id:</b> " + doc_ids).tolist()

edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode="lines",
    line=dict(width=1),
    hoverinfo="skip",
    name="k-NN edges",
    opacity=0.35
)

node_trace = go.Scatter3d(
    x=xyz[:, 0], y=xyz[:, 1], z=xyz[:, 2],
    mode="markers",
    marker=dict(size=3),
    text=hover_text,
    hoverinfo="text",
    name="Nodes"
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title="Interactive 3D HNSW / k-NN Graph (t-SNE projection)",
    scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    showlegend=False,
    margin=dict(l=0, r=0, t=40, b=0),
    height=780
)




## Graph Memory Usage (MB)

In [ ]:
import os
from pathlib import Path

INDEX_DIR = Path("vector_index_keywords_only")
paths = {
    "hnsw.index": INDEX_DIR / "hnsw.index",
    "meta.parquet": INDEX_DIR / "meta.parquet",
}

def mb(p: Path) -> float:
    return os.path.getsize(p) / (1024*1024) if p.exists() else 0.0

sizes = {name: mb(p) for name, p in paths.items()}
sizes["TOTAL_MB"] = sum(sizes.values())
sizes


{'hnsw.index': 1.1856403350830078,
 'meta.parquet': 0.0116424560546875,
 'TOTAL_MB': 1.1972827911376953}